In [21]:
import autogen
import config

In [22]:
# Import necessary libraries
import imaplib
import email
from email.header import decode_header
import re

# Define a function to fetch the latest email details from a specific sender
def fetch_latest_email_details(user, password, sender):
    # Connect to the Gmail IMAP server securely using SSL
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    # Log in to the specified email account
    mail.login(user, password)
    # Select the inbox for operations
    mail.select("inbox")

    # Search for emails from the specified sender, 'FROM' filter used in IMAP search command
    status, messages = mail.search(None, f'(FROM "{sender}")')
    # Check if the search was successful
    if status != "OK":
        print("No emails found!")
        return None, None

    # Extract the ID of the latest email from the specified sender
    latest_email_id = messages[0].split()[-1]
    # Fetch the entire email (using RFC822 protocol) corresponding to the latest ID
    status, data = mail.fetch(latest_email_id, "(RFC822)")
    if status != "OK":
        print("Failed to fetch the email.")
        return None, None

    # Parse the email content into a message object
    msg = email.message_from_bytes(data[0][1])
    # Decode the email subject, if it's encoded
    subject = decode_header(msg["subject"])[0][0]
    if isinstance(subject, bytes):
        # Decode to string if subject is in bytes
        subject = subject.decode()

    # Initialize body content variable
    body = ""
    # Check if the email is multipart (has multiple parts like body, attachments)
    if msg.is_multipart():
        # Iterate through each part of the email
        for part in msg.walk():
            # Get the content type of the part
            ctype = part.get_content_type()
            # Check if the part is plain text or HTML content
            if ctype == "text/plain" or ctype == "text/html":
                # Decode and assign the content to 'body' variable
                body = part.get_payload(decode=True).decode()
                break
    else:
        # For non-multipart emails, directly decode and assign content
        body = msg.get_payload(decode=True).decode()

    # Define regex patterns to find the quoted text that indicates a reply or forward
    patterns = [
        r"On\s+\w{3}\s+\d{1,2}\.\s+\w{3}\s+\d{4}\s+at\s+\d{1,2}:\d{2},\s+<[^>]+>\s+wrote:",  # Pattern 1
        r"On\s+\w{3},\s+\d{1,2}\s+\w{3}\s+\d{4},\s+\d{1,2}:\d{2}\s*,\s*<[^>]+>\s+wrote:",  # Pattern 2
        r"On\s+\w{3},\s+\d{1,2}\s+\w{3}\s+\d{4}\s+at\s+\d{1,2}:\d{2}\s+[AP]M,\s+<[^>]+>\s+wrote:"  # Pattern 3
    ]

    # Initialize a variable to track the first match position of the patterns
    first_match_pos = len(body)
    # Iterate through each pattern to find a match in the email body
    for pattern in patterns:
        match = re.search(pattern, body, re.MULTILINE | re.IGNORECASE | re.DOTALL)
        # If a match is found and its position is before any previous match, update 'first_match_pos'
        if match and match.start() < first_match_pos:
            first_match_pos = match.start()

    # Extract everything before the first match as the latest message, otherwise consider the entire body
    latest_message = body[:first_match_pos].strip() if first_match_pos != len(body) else body

    # Return the decoded subject and the latest message content
    return subject, latest_message

In [24]:
# Set Gmail credentials and sender's email address
user = config.user_email
password = config.app_password
sender_email = config.sender

# Fetch latest email details (subject and body) from specified sender
subject, latest_email_body = fetch_latest_email_details(user, password, sender_email)

# Define file path for saving email content
file_path = "latest_email.txt"

# Save email content to file if fetched successfully
if subject and latest_email_body:
    # Write email subject and body to file if retrieval was successful
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(f"Subject: {subject}\n\n{latest_email_body}")
    print(f"Email saved to {file_path}")
else:
    # Print failure message if email fetching fails
    print("Failed to fetch email or no emails from the specified sender.")

Email saved to latest_email.txt


In [10]:
# Define OpenAI API configuration
config_list = [
    {"model": "gpt-3.5-turbo", "api_key": config.api}
]

# Create an AssistantAgent with LLM configuration for generating email responses
email_response_assistant = autogen.AssistantAgent(
    name="EmailResponseAssistant",
    llm_config={
        "cache_seed": 42,  # Set a cache seed for consistent responses
        "config_list": config_list,  # Apply the API configuration
        "temperature": 0.7,  # Adjust creativity/variability of responses
    },
)


# Initialize the UserProxyAgent for simulating user interactions
email_user_proxy = autogen.UserProxyAgent(
    name="EmailUserProxy",
    human_input_mode="NEVER", # No human input; fully automated
    max_consecutive_auto_reply=1,  # Limit on auto-replies to prevent loops
    code_execution_config={
        "work_dir": "email_handling", # Working directory for operations
        "use_docker": False,  # Consider enabling Docker for safer code execution if available
    }
)

# Function to read an email's subject and body from a file
def read_email_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        subject = file.readline().strip() # Read the first line as the subject
        email_body = file.read().strip() # Read the rest as the email body
    return subject, email_body


# Generate a response for an email given its subject and body
def generate_email_response(subject, email_body):
    # Construct a prompt for the AssistantAgent
    detailed_prompt = f"Please read the following email and respond politely and concisely.\n\n{subject}\n\n{email_body}"
    # Initiate a chat with the AssistantAgent using the UserProxyAgent
    chat_res = email_user_proxy.initiate_chat(
        recipient=email_response_assistant,
        message=detailed_prompt,
        summary_method="reflection_with_llm",  # Summary method configuration
    )
    return chat_res  # Return the generated chat response


# Main operations
# Main execution flow
input_file_path = "latest_email.txt"  # Path to the email file
subject, email_body = read_email_from_file(input_file_path)  # Read email content
response_content = generate_email_response(subject, email_body)  # Generate response


# Output the generated email response
print(f"Generated Email Response:\n{response_content}")

EmailUserProxy (to EmailResponseAssistant):

Please read the following email and respond politely and concisely.

Subject: Meeting Scheduling Request

Hi Shashi,

I hope you're well. I'd like to schedule a meeting to discuss our
next project. Could we meet on 29 March, at 1:30 am? It should take 30
minutes. Let me know if this works or suggest an alternative.

Best regards,
Shila

--------------------------------------------------------------------------------
EmailResponseAssistant (to EmailUserProxy):

```python
# No specific information to gather.
# Let's draft a response to the email.

print("Subject: Re: Meeting Scheduling Request\n")
print("Hi Shila,\n")
print("I hope you're doing well. Thank you for reaching out to schedule a meeting to discuss our next project.\n")
print("Unfortunately, the proposed timing of 1:30 am on 29 March is a bit late for me. Could we possibly reschedule to a time during regular working hours?")
print("Looking forward to our discussion.\n")
print("Best 

In [12]:
def extract_final_draft_email_from_chat_result(chat_result):
    final_draft_email = None
    email_identifiers = ["To:", "Subject:"]
    # Ensure chat_history is accessible and is a list; otherwise, return None
    if hasattr(chat_result, 'chat_history') and isinstance(chat_result.chat_history, list):
        # Reverse the list to start from the end
        for message in reversed(chat_result.chat_history):
            # Check if the role is 'assistant' or 'user' and content is not empty or just "TERMINATE"
            if message['role'] in ['assistant', 'user'] and any(identifier in message['content'] for identifier in email_identifiers) and message['content'].strip().lower() != "terminate" and message['content'].strip():
                final_draft_email = message['content']
                break
    return final_draft_email

# Use the function with your response_content object
final_draft_email = extract_final_draft_email_from_chat_result(response_content)

# Print the final draft email if found
if final_draft_email:
    print("Final Draft Email:\n", final_draft_email)
else:
    print("No final draft email found.")


Final Draft Email:
 exitcode: 0 (execution succeeded)
Code output: 
Subject: Re: Meeting Scheduling Request

Hi Shila,

I hope you're doing well. Thank you for reaching out to schedule a meeting to discuss our next project.

Unfortunately, the proposed timing of 1:30 am on 29 March is a bit late for me. Could we possibly reschedule to a time during regular working hours?
Looking forward to our discussion.

Best regards,
Shashi



In [13]:
def process_final_draft_email(final_email_draft, new_signature):
    # Define end markers for the email signature
    end_markers = [
        'Best regards,', 'With regards,', 'Best,', 'Regards', "Sincerely,", "Yours truly,", 
        "Yours sincerely,", "Thank you,", "Warm regards,", "Kind regards,", "Respectfully,", 
        "Yours faithfully,", "Cheers,", "Yours,"
    ]  # Markers that indicate the start of the signature

    sender_name_placeholder = " [Sender's Name]"  # Placeholder to be replaced
    
    # Initialize variables
    extracted_subject = ""
    extracted_content = ""
    body_started = False
    body_lines = []
    lines = final_email_draft.split('\n')
    
    # Process each line to extract the subject and the body
    for line in lines:
        if line.startswith("Subject:"):
            extracted_subject = line.strip()  # Extract the subject
            body_started = True  # Indicate that the next lines should be considered as the body
        elif body_started:
            # Check if the line contains any of the signature end markers
            if any(marker in line for marker in end_markers):
                # Stop adding to body_lines when the first end marker is found
                # Replace the entire line with the new signature
                body_lines.append(new_signature)
                break
            else:
                if sender_name_placeholder in line:
                    line = line.replace(sender_name_placeholder, "")
                body_lines.append(line)
    
    # Join the lines to form the extracted body content
    extracted_content = '\n'.join(body_lines)
    
    return extracted_subject, extracted_content

# Define the new signature
new_signature = "Best regards,\nShashi Sharma"

# Process the final draft email
extracted_subject, modified_body = process_final_draft_email(final_draft_email, new_signature)

# Print the processed parts
print(extracted_subject)
print("\n" + modified_body)

Subject: Re: Meeting Scheduling Request


Hi Shila,

I hope you're doing well. Thank you for reaching out to schedule a meeting to discuss our next project.

Unfortunately, the proposed timing of 1:30 am on 29 March is a bit late for me. Could we possibly reschedule to a time during regular working hours?
Looking forward to our discussion.

Best regards,
Shashi Sharma


In [27]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Gmail SMTP server configuration
smtp_server = 'smtp.gmail.com'
smtp_port = 587  # Use 587 for TLS

# update Sender and recipient
updatted_sender_email = user  # Sending from your Gmail account
updated_recipient_email = sender_email  # The original sender's email you're replying to


# Create message
message = MIMEMultipart()
message['From'] = updatted_sender_email
message['To'] = updated_recipient_email
message['Subject'] = extracted_subject.replace("Subject: ", "Re:", 1)  # Remove the "Subject: " prefix if present

# Attach the email content
message.attach(MIMEText(modified_body, 'plain'))

# Send the email
try:
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()  # Upgrade the connection to secure
        server.login(user, password)
        server.sendmail(updatted_sender_email, updated_recipient_email, message.as_string())
        print("Email sent successfully!")
except Exception as e:
    print(f"Failed to send email: {e}")

Email sent successfully!
